
# Office in Pygame (Non-Default Gym Environment)

Let's explore creating a custom gym environment. We'll still use a game as our base, but you should note how this could really be any program with inputs and outputs.

In [ ]:
import pygame, sys, time, random
from pygame.surfarray import array3d
import random
import pickle
## Sets up colors for the game using RGB Codes
BLACK = pygame.Color(0, 0, 0)
WHITE = pygame.Color(255, 255, 255)
RED = pygame.Color(255, 0, 0)
GREEN = pygame.Color(0, 255, 0) 
with open('seats.pkl', 'rb') as f:
    seats_coordinates = pickle.load(f)


In [ ]:
class OfficeEnv():
    
    def __init__(self,frame_size_x,frame_size_y):
        '''
        Defines the initial game window size
        '''
        self.frame_size_x = frame_size_x
        self.frame_size_y = frame_size_y
        self.game_window = pygame.display.set_mode((self.frame_size_x, self.frame_size_y))        
        self.reset()
    
    def reset(self):
        '''
        Resets the game, along with the default Office size and spawning desktop.
        '''
        #self.game_window.fill(BLACK)


        self.Office_pos = [100, 50]
        self.Office_body = [[100, 50], [100-10, 50], [100-(2*10), 50]]
        self.desktop_pos = self.spawn_desktop()
        
        self.desktop_spawn = True

        self.direction = "RIGHT"
        self.action = self.direction
        self.score = 0
        self.steps = 0
        print("Game Reset.")
    
    def change_direction(self,action,direction):
        '''
        Changes direction based on action input. 
        Checkes to make sure Office can't go back on itself.
        '''
        
        if action == 'UP' and direction != 'DOWN':
            direction = 'UP'
        if action == 'DOWN' and direction != 'UP':
            direction = 'DOWN'
        if action == 'LEFT' and direction != 'RIGHT':
            direction = 'LEFT'
        if action == 'RIGHT' and direction != 'LEFT':
            direction = 'RIGHT'
            
        return direction
    
    def move(self,direction,Office_pos):
        '''
        Updates Office_pos list to reflect direction change.
        '''
            
        if direction == 'UP':
            Office_pos[1] -= 10
        if direction == 'DOWN':
            Office_pos[1] += 10
        if direction == 'LEFT':
            Office_pos[0] -= 10
        if direction == 'RIGHT':
            Office_pos[0] += 10
            
        return Office_pos
    
    def eat(self):
        '''
        Returns Boolean indicating if Office has "eaten" the white desktop square
        '''
        return self.Office_pos[0] == self.desktop_pos[0] and self.Office_pos[1] == self.desktop_pos[1]
    
   
    def spawn_desktop(self):
        '''
        Spawns desktop in a random location on window size
        '''

        
        #desktop_pos_1=random.choice(seats_coordinates)
        #return[desktop_pos_1[0], desktop_pos_1[1]]
        return [random.randrange(1, (self.frame_size_x//10)) * 10, random.randrange(1, (self.frame_size_y//10)) * 10]
    
    def human_step(self,event):   
        '''
        Takes human keyboard event and then returns it as an action string
        '''
        
        action = None
        
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        ################################################ 
        ########## CONVERT KEYPRESS TO DIRECTION ###### 
        ############################################## 
        elif event.type == pygame.KEYDOWN:
            
            if event.key == pygame.K_UP:
                action = 'UP'
            if event.key == pygame.K_DOWN:
                action = 'DOWN'
            if event.key == pygame.K_LEFT:
                action = 'LEFT'
            if event.key == pygame.K_RIGHT:
                action = 'RIGHT'
            # Esc -> Create event to quit the game
            if event.key == pygame.K_ESCAPE:
                pygame.event.post(pygame.event.Event(pygame.QUIT))
                
        return action
    
    
    
    def display_score(self,color, font, size):
        '''
        Updates the score in top left
        '''
        score_font = pygame.font.SysFont(font, size)
        score_surface = score_font.render('Score : ' + str(self.score), True, color)
        score_rect = score_surface.get_rect()
        score_rect.midtop = (self.frame_size_x/10, 15)
        self.game_window.blit(score_surface, score_rect)
        
    def game_over(self):
        '''
        Checks if the Office has touched the bounding box or itself
        '''
        
        # TOUCH BOX
        if self.Office_pos[0] < 0 or self.Office_pos[0] > self.frame_size_x-10:
            self.end_game()
        if self.Office_pos[1] < 0 or self.Office_pos[1] > self.frame_size_y-10:
            self.end_game()

        # TOUCH OWN BODY
        for block in self.Office_body[1:]:
            if self.Office_pos[0] == block[0] and self.Office_pos[1] == block[1]:
                self.end_game()
 


    def end_game(self):
        '''
        
        '''
        message = pygame.font.SysFont('arial', 45)
        message_surface = message.render('Game has Ended.', True, RED)
        message_rect = message_surface.get_rect()
        message_rect.midtop = (self.frame_size_x/2, self.frame_size_y/4)
        self.game_window.fill(BLACK)
        self.game_window.blit(message_surface, message_rect)
        self.display_score(RED, 'times', 20)
        pygame.display.flip()
        time.sleep(3)
        pygame.quit()
        sys.exit()

Then we execute the game

In [ ]:
Office_env = OfficeEnv(1024,768)

# This is technically a FPS Refresh rate
# Higher number means faster refresh, thus faster Office movement, meaning harder game play
difficulty = 10


# FPS (frames per second) controller
fps_controller = pygame.time.Clock()

# Checks for errors encountered
check_errors = pygame.init()


# Initialise game window
pygame.display.set_caption('Office Eater') 




#The main game loop
running = True
while running:
    
    # Check Input from Human Step 
    for event in pygame.event.get():
        
        Office_env.action = Office_env.human_step(event)
    
        #if event.type == pygame.QUIT:
        #    running = False
   


    # Check for Direction change based on action
    Office_env.direction = Office_env.change_direction(Office_env.action,Office_env.direction)


    #Update Office Position
    Office_env.Office_pos = Office_env.move(Office_env.direction,Office_env.Office_pos)


    # Check to see if we ate some desktop
    Office_env.Office_body.insert(0, list(Office_env.Office_pos))
    if Office_env.eat():
        Office_env.score += 1
        Office_env.desktop_spawn = False
    else:
        Office_env.Office_body.pop()

    # Check to see if we need to spawn new desktop 
    if not Office_env.desktop_spawn:
        Office_env.desktop_pos = Office_env.spawn_desktop()
    Office_env.desktop_spawn = True

    # Draw the Office
   # Office_env.game_window.fill(BLACK)

    bg = pygame.image.load("background.png")
    #INSIDE OF THE GAME LOOP
    Office_env.game_window.blit(bg, (0, 0))    
        
    
    
    for pos in Office_env.Office_body:
        pygame.draw.rect(Office_env.game_window, GREEN, pygame.Rect(pos[0], pos[1], 10, 10))
        
    # Draw desktop
    pygame.draw.rect(
        
                     Office_env.game_window, RED, 
                     pygame.Rect(Office_env.desktop_pos[0], 
                     Office_env.desktop_pos[1], 10, 10)
        
                     )

    # Check if we lost
    Office_env.game_over()
    
    

    Office_env.display_score(BLACK, 'consolas', 20)
    # Refresh game screen
    pygame.display.update()
    # Refresh rate
    fps_controller.tick(difficulty)
    img = array3d(Office_env.game_window)

In [ ]:
self.image = QLabel()
self.image.setPixmap(QPixmap(path))
self.image.setObjectName("image")
self.image.mousePressEvent = self.getPos

def getPos(self , event):
    x = event.pos().x()
    y = event.pos().y()